In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
import urllib

In [5]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
urllib.request.urlretrieve(weights_url, 'inception_v3.h5')

('inception_v3.h5', <http.client.HTTPMessage at 0x251bdb1b760>)

In [31]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'horse-or-human/training/',
    target_size=(150,150),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [32]:
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_generator = validation_datagen.flow_from_directory(
    'horse-or-human/validation/',
    target_size=(150,150),
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [25]:
model = InceptionV3(input_shape=(150,150,3), include_top=False, weights=None)

In [26]:
model.load_weights('inception_v3.h5')
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 74, 74, 32)   864         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_282 (Batch  (None, 74, 74, 32)  96          ['conv2d_282[0][0]']             
 Normalization)                                                                                   
                                                                                       

In [27]:
for layer in model.layers:
    layer.trainable = False

last_output = model.get_layer('mixed7').output

In [28]:
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

In [29]:
model = Model(model.input,x)
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])

In [33]:
model.fit(train_generator, epochs=40, validation_data= validation_generator)

Epoch 1/40
33/33 [==============================] - 40s 944ms/step - loss: 0.2259 - acc: 0.9309 - val_loss: 0.0942 - val_acc: 0.9688
Epoch 2/40
33/33 [==============================] - 30s 880ms/step - loss: 0.0286 - acc: 0.9854 - val_loss: 0.2018 - val_acc: 0.9258
Epoch 3/40
33/33 [==============================] - 30s 910ms/step - loss: 0.0198 - acc: 0.9903 - val_loss: 0.4542 - val_acc: 0.8633
Epoch 4/40
33/33 [==============================] - 28s 854ms/step - loss: 0.0214 - acc: 0.9912 - val_loss: 0.4510 - val_acc: 0.9062
Epoch 5/40
33/33 [==============================] - 28s 832ms/step - loss: 0.0062 - acc: 0.9981 - val_loss: 0.1414 - val_acc: 0.9492
Epoch 6/40
33/33 [==============================] - 34s 1s/step - loss: 0.0125 - acc: 0.9971 - val_loss: 0.2087 - val_acc: 0.9414
Epoch 7/40
33/33 [==============================] - 31s 931ms/step - loss: 0.0112 - acc: 0.9990 - val_loss: 0.3765 - val_acc: 0.9336
Epoch 8/40
33/33 [==============================] - 37s 1s/step - loss: 